In [1]:
import synthetic;
import json
pathToMotifs = "motifs.txt";
loadedMotifs = synthetic.LoadedEncodeMotifs(pathToMotifs, pseudocountProb=0.001)

ctcfSampler = synthetic.PwmSamplerFromLoadedMotifs(loadedMotifs,motifName="CTCF_known1")


In [2]:
spi1Sampler = synthetic.BestHitPwmFromLoadedMotifs(loadedMotifs,motifName="SPI1_known1")

singleCtcfEmbedder = synthetic.SubstringEmbedder(substringGenerator=ctcfSampler
                                                ,positionGenerator=synthetic.UniformPositionGenerator())

In [3]:
singleSpi1Embedder = synthetic.SubstringEmbedder(substringGenerator=spi1Sampler
                                                ,positionGenerator=synthetic.OutsideCentralBp(400))

truncatedPoissonGenerator=synthetic.MinMaxWrapper(quantityGenerator=synthetic.PoissonQuantityGenerator(3)
                                                    ,theMin=1,theMax=5)

In [4]:
zeroInflater = synthetic.ZeroInflater(quantityGenerator=synthetic.PoissonQuantityGenerator(5), zeroProb=0.5)

In [5]:
singleIRFembedder = synthetic.SubstringEmbedder(
                substringGenerator=synthetic.BestHitPwmFromLoadedMotifs(loadedMotifs,motifName="IRF_known1")
                ,positionGenerator=synthetic.UniformPositionGenerator())

embedInBackground = synthetic.EmbedInABackground(
    backgroundGenerator=synthetic.ZeroOrderBackgroundGenerator(seqLength=500) 
    , embedders=[
        #synthetic.RepeatedEmbedder(
        #    embedder=singleCtcfEmbedder
        #    ,quantityGenerator=truncatedPoissonGenerator
        #),
        synthetic.XOREmbedder(
            embedder1=singleSpi1Embedder
            ,embedder2=singleIRFembedder
            ,probOfFirst=0.5 
        )
    ]
);

In [6]:
sequenceSet = synthetic.GenerateSequenceNTimes(embedInBackground, 10)
print json.dumps(sequenceSet.getJsonableObject(), indent=4)

{
    "numSeq": 10, 
    "sequenceGenerator": {
        "class": "EmbedInABackground", 
        "namePrefix": "synth", 
        "backgroundGenerator": "zeroOrderBackground-500", 
        "embedders": [
            {
                "class": "XOREmbedder", 
                "embedder1": {
                    "substringGenerator": {
                        "motifName": "SPI1_known1", 
                        "pwmSubstringGenerator": "bestHit-SPI1_known1", 
                        "loadedMotifs": {
                            "fileName": "motifs.txt", 
                            "pseudocountProb": 0.001
                        }
                    }, 
                    "positionGenerator": "outsideCentral-400"
                }, 
                "embedder2": {
                    "substringGenerator": {
                        "motifName": "IRF_known1", 
                        "pwmSubstringGenerator": "bestHit-IRF_known1", 
                        "loadedMotifs": {
                   

In [7]:
for generatedSeq in sequenceSet.generateSequences():
    print generatedSeq.seqName, generatedSeq.seq, " | ".join(str((x.seq, x.startPos)) for x in generatedSeq.embeddings)

synth1 CGGTGTTTCAGCTCTGAAAGAGGAAGTAACATATGTCTAGACATAATCGGGACGCCAATTCATGCTGGTGCAAATGGTTAATATTATGGACGATTGTTTTGCATATCTGGGTCGTCATAAGGACTTTGTATAACCATAGCTTTAAGAGATTAGGTTTTTGCATTTTGTTCGGATACCAGAGAGCAACTATTCAGGGTGTTCTGGAAAATGTATTTAGATAATATATCCCATTTAGGACAGTCGTGCGGCATTTAGCGTCCTTAGCACAAAGCAAACAATCGCGCATTATGCTATCATTACTTTAATACTCTTACACTCATCCCGAATCCTTCGGCTATTTGCAAAAGTACGACTGGCTTTATGCCACCACTCGACCATACCTATTATTCTCTTTTTGTAATTAAACTTCAGTCTACCACGGTCGTGTTCGTAAGTGAGTAGTGATAAAGAAGTTCAAGACTCGCAATATCATACTAGCCCTAATCAAATATTTACATAGC ('AGAGGAAG', 18)
synth2 ATCCCATGTCACATTTTTCCTAACGCACTCCGTAACTTCCCGTATACAGCTCACATAGCGTATGCAAGGCGCAAACGACTATCTTATTATATTAAATCGTTAGTCTGTTGCATACACATTCTTATTTACTCCCCTCAATTGATAGCGCATAAGCGATCGTCTAATCTATACTAAATGTCTGCCTGTGCGATTTTATTTGAGAATGGTCCTGGGGAGAACAAACATTTCTCATGGTTAGTGATCGTCTCATCTAGGAAATCAGTTAGTCAGCCTAAAGTATAGTGGAAGTAGCAACGCCATCCTAGATCCATACCACAGTGTAAATACCGTGATATGTGCGAGATTAATGGAGTCTTTTAAAGCGGATCTAGCCGCTAAGTCTGTCCGTCCCTTTTCTTGAGATTGGCATATGGTCGACTAACAACTTTAAATTTGAAGCTTAGGCAACTCCACTATCTCGGTCGACGA

In [8]:
str(loadedMotifs.__class__)

"<class 'synthetic.LoadedEncodeMotifs'>"